## Cat Facts
This notebook shows how to create, run, and use output from the Cat Facts assistant! This assistant uses a free demonstration web service with no authentication.

The assistant definition enables Code Interpreter which the assistant uses to create a file which is downloaded in the last step.

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.8.6"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.8.6

## List the assistants in your deployment
If you get an error, verify your setup!

In [4]:
var assistants = (await client.AssistantList()).Data;
foreach(var assistant in assistants)
{
    Console.WriteLine(assistant.Name);
}

ConversationUserProxy
CopilotExtensions
TCEnergy4o-mini
TCEnergy4o
NoteAntsTest
Blob Pirate
OneNote Ants
Web Search Function Demo
TranscriptAnts
McKessonPDFTranscripts
SillyPirate
AssistAnts
Web Search Ants
Python Ants
Outlook Ants


In [3]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Cat Facts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find cat facts");
}

Deleted assistant


## Create the Cat Facts assistant
The definition of this assistant is located in the ".\AssistantDefinitions\Cat Facts" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Cat Facts" contains the following files:
```
Cat Facts
│   instructions.md - The assistant's instructions. This file is optional but when present overwrites the value of the instructions property in manifest.json
│   manifest.json - The assistants base settings including model, "model": "gpt-4o". You need to ensure you have deployed whatever models your assistants require
│
└───OpenAPI
        cat-fact.herokuapp.com.json - The swagger definition for the test service

```

[See https://github.com/alexwohlbruck/cat-facts](https://github.com/alexwohlbruck/cat-facts)

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the cat-facts swagger which follows an Open API schema into tool defintions for the assistant.

In [5]:
var assistantId = await AssistantUtility.Create("Cat Facts", config);
Console.WriteLine(assistantId)

asst_bVo30Rj6V4nHK8gYlN8yfYah


## Run the Assistant

`output.Dialog` shows the conversation. This version of AntRunner uses a built-in assistant definition. Upon completion the thread is deleted.

In [6]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Cat Facts",
    Instructions = "Create a plan to tell me something about cats. Review with me before using any tools",
    UseConversationEvaluator = true
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");


User: Create a plan to tell me something about cats. Review with me before using any tools

Assistant: Sure! Here's a plan to tell you something about cats:

1. **Retrieve Cat Facts**: 
   - Use the `GetCatFacts` function to retrieve a list of interesting facts about cats.

2. **Review and Share Facts**:
   - Review the retrieved facts to ensure they are interesting and relevant.
   - Share a selection of these facts with you.

Would you like to proceed with this plan, or do you have any specific requests or adjustments to make?

User: Yes, go ahead

Assistant: I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.","__v":0,"source":"user","updatedAt":"2020-08-26T20:20:02.359Z","type":"cat","createdAt":"2018-01-14T21:20:02.750Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e008ad0aac31001185ed0c","user":"58e007480aac31001185ecef","text":"The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.","__v":0,"source":"user","updatedAt":"2020-08-24T20:20:01.867Z","type":"cat","createdAt":"2018-03-15T20:20:03.281Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e007cc0aac31001185ecf5","user":"58e007480aac31001185ecef","text":"Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-01T21:20:02.713Z","deleted":false,"used":false}]
```

Here are some interesting facts about cats that I found:

1. **Health Benefits**:
   - Owning a cat can reduce the risk of stroke and heart attack by a third.

2. **Dietary Note**:
   - Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat clean, cool drinking water.

3. **Daily Routine**:
   - Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.

4. **Healing Abilities**:
   - The frequency of a domestic cat's purr is the same frequency at which muscles and bones repair themselves.

5. **Popularity**:
   - Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.

Would you like more details on any of these facts, or is there anything else specific you would like to know about cats?


## output.LastMessage
A core perspective of AntRunnerLib is to get the richness of the assistants API with file retrieval, code interpreter, and function calls in the logical form of a chat completion.
```
output.LastMessage // Final answer
```


In [7]:
output.LastMessage.DisplayAs("text/markdown");

Here are some interesting facts about cats that I found:

1. **Health Benefits**:
   - Owning a cat can reduce the risk of stroke and heart attack by a third.

2. **Dietary Note**:
   - Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat clean, cool drinking water.

3. **Daily Routine**:
   - Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.

4. **Healing Abilities**:
   - The frequency of a domestic cat's purr is the same frequency at which muscles and bones repair themselves.

5. **Popularity**:
   - Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.

Would you like more details on any of these facts, or is there anything else specific you would like to know about cats?

## Downloading files from code interpreter
In addition to the Cat Facts API, this assistant also uses code interpreter based on its [manifest.json](.\AssistantDefinitions\Cat%20Facts\manifest.json):

```json
{
  "name": "Cat Facts",
  "description": "I look up facts about cats and can also write Python code!",
  "instructions": "",
  "tools": [
    { "type": "code_interpreter" }
  ],
  "tool_resources": {},
  "top_p": 0.95,
  "response_format": "auto",
  "metadata": {
  },
  "model": "gpt-4o",
  "temperature": 0.7
}
```

This prompt instructs the assistant to create a file **"Tell me something about cats and put the results into a file I can download"**. 

As a result the output contains Annotations. You can download the ones with annotation.Type == "file_path" as demonstrated below.

In [8]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Cat Facts",
    Instructions = "Tell me something about cats and put the results into a file I can download",
    UseConversationEvaluator = false // Turn off the conversation evaluator
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
output.Dialog.DisplayAs("text/markdown");
Console.WriteLine(output.Annotations[0].FilePathAnnotation.FileId);

// Download files specified in FilePathAnnotation
foreach (var annotation in output.Annotations)
{
    if (annotation.Type == "file_path")
    {
        var fileName = await annotation.GetFileName(config);

        var filePath = Path.Combine(@".\", fileName);
        await annotation.DownloadFile(filePath, config);
        Console.WriteLine($"{filePath}");
    }
}



User: Tell me something about cats and put the results into a file I can download

Assistant: I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.","__v":0,"source":"user","updatedAt":"2020-08-26T20:20:02.359Z","type":"cat","createdAt":"2018-01-14T21:20:02.750Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e008ad0aac31001185ed0c","user":"58e007480aac31001185ecef","text":"The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.","__v":0,"source":"user","updatedAt":"2020-08-24T20:20:01.867Z","type":"cat","createdAt":"2018-03-15T20:20:03.281Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e007cc0aac31001185ecf5","user":"58e007480aac31001185ecef","text":"Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-01T21:20:02.713Z","deleted":false,"used":false}]
```

I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.","__v":0,"source":"user","updatedAt":"2020-08-26T20:20:02.359Z","type":"cat","createdAt":"2018-01-14T21:20:02.750Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e008ad0aac31001185ed0c","user":"58e007480aac31001185ecef","text":"The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.","__v":0,"source":"user","updatedAt":"2020-08-24T20:20:01.867Z","type":"cat","createdAt":"2018-03-15T20:20:03.281Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e007cc0aac31001185ecf5","user":"58e007480aac31001185ecef","text":"Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-01T21:20:02.713Z","deleted":false,"used":false}]
```

I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.","__v":0,"source":"user","updatedAt":"2020-08-26T20:20:02.359Z","type":"cat","createdAt":"2018-01-14T21:20:02.750Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e008ad0aac31001185ed0c","user":"58e007480aac31001185ecef","text":"The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.","__v":0,"source":"user","updatedAt":"2020-08-24T20:20:01.867Z","type":"cat","createdAt":"2018-03-15T20:20:03.281Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e007cc0aac31001185ecf5","user":"58e007480aac31001185ecef","text":"Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-01T21:20:02.713Z","deleted":false,"used":false}]
```

I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.","__v":0,"source":"user","updatedAt":"2020-08-26T20:20:02.359Z","type":"cat","createdAt":"2018-01-14T21:20:02.750Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e008ad0aac31001185ed0c","user":"58e007480aac31001185ecef","text":"The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.","__v":0,"source":"user","updatedAt":"2020-08-24T20:20:01.867Z","type":"cat","createdAt":"2018-03-15T20:20:03.281Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e007cc0aac31001185ecf5","user":"58e007480aac31001185ecef","text":"Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-01T21:20:02.713Z","deleted":false,"used":false}]
```

I called the tool named GetCatFacts with these arguments:
```
{}
```
and got this result:
```
[{"status":{"verified":true,"sentCount":1},"_id":"58e008780aac31001185ed05","user":"58e007480aac31001185ecef","text":"Owning a cat can reduce the risk of stroke and heart attack by a third.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-29T20:20:03.844Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"58e009390aac31001185ed10","user":"58e007480aac31001185ecef","text":"Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-04T21:20:02.979Z","deleted":false,"used":false},{"status":{"verified":true,"sentCount":1},"_id":"588e746706ac2b00110e59ff","user":"588e6e8806ac2b00110e59c3","text":"Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.","__v":0,"source":"user","updatedAt":"2020-08-26T20:20:02.359Z","type":"cat","createdAt":"2018-01-14T21:20:02.750Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e008ad0aac31001185ed0c","user":"58e007480aac31001185ecef","text":"The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.","__v":0,"source":"user","updatedAt":"2020-08-24T20:20:01.867Z","type":"cat","createdAt":"2018-03-15T20:20:03.281Z","deleted":false,"used":true},{"status":{"verified":true,"sentCount":1},"_id":"58e007cc0aac31001185ecf5","user":"58e007480aac31001185ecef","text":"Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs.","__v":0,"source":"user","updatedAt":"2020-08-23T20:20:01.611Z","type":"cat","createdAt":"2018-03-01T21:20:02.713Z","deleted":false,"used":false}]
```

I ran this code:
```
cat_facts = [
    "Owning a cat can reduce the risk of stroke and heart attack by a third.",
    "Most cats are lactose intolerant, and milk can cause painful stomach cramps and diarrhea. It's best to forego the milk and just give your cat the standard: clean, cool drinking water.",
    "Domestic cats spend about 70 percent of the day sleeping and 15 percent of the day grooming.",
    "The frequency of a domestic cat's purr is the same at which muscles and bones repair themselves.",
    "Cats are the most popular pet in the United States: There are 88 million pet cats and 74 million dogs."
]

file_path = '/mnt/data/cat_facts.txt'

with open(file_path, 'w') as file:
    for fact in cat_facts:
        file.write(fact + '\n')

file_path
```

I have compiled some interesting facts about cats into a file. You can download it using the link below:

[Download Cat Facts](sandbox:/mnt/data/cat_facts.txt)


assistant-NnDdZFQAB7V4IpzVfRy5d9M8
.\cat_facts.txt


## Clean up

In [11]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Cat Facts");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find Cat Facts");
}

Deleted assistant
